In [1]:
def timetable(date1, date2 = 'not_date', to_json = False):
    import datetime
    from progress.bar import IncrementalBar
    list_b = list(map(int, date1.split('-')))
    date_b = datetime.date(list_b[0],list_b[1],list_b[2])
    if date2 == 'not_date':
        date_a = date_b
    else:
        list_a = list(map(int, date2.split('-')))
        date_a = datetime.date(list_a[0],list_a[1],list_a[2])
    import requests
    import time
    from bs4 import BeautifulSoup
    base_url = 'https://kinoteatr.ru/kinoafisha/?date='
    dic = {}
  
    for i in range((date_a - date_b).days + 1):
        r = requests.get(f'{base_url}{date_b}')
        soap = BeautifulSoup(r.text, 'lxml')
        dic[date_b] = {}
        for block in soap.findAll('div', class_='col-md-2 col-sm-6 col-xs-12 movie_card'):
            href = block.find('a', class_='movie_card_clickable_zone gtm-ec-list-item-movie')['href']
            name = block.find('span', class_='movie_card_header title').text.strip()
            dic[date_b][name] = {}
            film_r = requests.get(href)
            film_soap = BeautifulSoup(film_r.text, 'lxml')
            for tt in film_soap.findAll('div', class_='cinema'):
                c_name = tt.find('div', class_='name').text.strip()
                lsttime = []
                for time in tt.findAll('div', class_='time'):
                    t = time.text.strip()[0:5]
                    lsttime.append(t)
                dic[date_b][name][c_name] = lsttime
        date_b = date_b + datetime.timedelta(days=1)
    if to_json:
        with open('{}.json'.format(to_json), 'w') as f:
            json.dump(dic, f)
        return True 
    else:
        return dic
timetable('2020-11-26','2020-11-30','kino')


ConnectionError: HTTPSConnectionPool(host='kinoteatr.ru', port=443): Max retries exceeded with url: /film/na-ostrie-2/?date=2020-11-28 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001954BB176A0>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))